# TikTok

In [101]:
# pip install PyTikTokAPI
from TikTokAPI import TikTokAPI
import pandas as pd
import numpy as np
import math
import requests
import nest_asyncio
nest_asyncio.apply()
import pyppeteer
import pyppeteer_stealth
import time

In [231]:
sheet_id = ''
sheet_name = ''
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}' 
data_from_excel = pd.read_csv(url)

In [233]:
# auto-generate key to join later!

data_from_excel.reset_index(inplace=True)

In [237]:
# Preprocessing data
data.dropna(subset=["Link Video"], inplace=True)
Tiktok_data = data[data['Link Video'].str.startswith('https://www.tiktok.com/')]

# get rid of unwanted new line and spacing
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)

# get rid of '?...'
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.split("?", expand=True)[0]

# Some links are from tiktoke main page
Tiktok_data = Tiktok_data[Tiktok_data['Link Video'].str.contains("/video/")]

<ipython-input-237-ccbfe1deb0ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
<ipython-input-237-ccbfe1deb0ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)
<ipython-input-237-ccbfe1deb0ea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [239]:
cookie = {
  "s_v_web_id": "verify_kse1hozq_sL9KtG3W_wm1J_43ZU_8zFB_P0AlRDxsUSIO",
  "tt_webid": "6989095610934822402"
}

api = TikTokAPI(cookie=cookie)

In [240]:
import datetime
begin_time = datetime.datetime.now()

dict_list = [] # Store all the info
viewcount_list = [] # Store view counts

for link in Tiktok_data['Link Video'].values:
    video_id = int(link[-19:]) # Take the Id part

    try:
        retval = api.getVideoById(video_id)
        dict_list.append(retval)
        
        view = retval['itemInfo']['itemStruct']['stats']['playCount']
        
        item = {    
                    "Link Video": link,
                    "Số View": view,
        }
        
        viewcount_list.append(item)
        
    except:
        print("Fail!" + link)
    
df = pd.DataFrame(viewcount_list)
df = Tiktok_data.merge(df, on="Link Video", how="left")
df.to_pickle("./tiktok.pkl")

print(datetime.datetime.now() - begin_time)

Fail!https://www.tiktok.com/@thuycute05123/video/6985715429998185754
Fail!https://www.tiktok.com/@thuonghienn/video/6996620377145232666
Fail!https://www.tiktok.com/@meous99/video/6997343418158419227
0:08:14.249778


In [241]:
Tiktok_datetime = datetime.datetime.now()
print(Tiktok_datetime)

2021-08-18 15:53:26.247779


In [ ]:
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

# Instagram

In [243]:
# Importing packages
from selenium import webdriver
# Give access to Enter & ESC keys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import json

In [246]:
insta_data = data[data['Link Video'].str.contains('instagram')]

# get rid of ?utm_...
insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]

<ipython-input-246-f17724073f65>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]


In [247]:
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/accounts/login/")

<ipython-input-247-84a1d52b201d>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [248]:
insta_list = []

for link in insta_data["Link Video"].values:
    base_url = link
    url_json = (base_url + '?__a=1')
    driver.get(url_json)
    
    data_j = driver.find_element_by_tag_name('body').text
    try:
        data_json = json.loads(data_j)

        view = data_json["graphql"]['shortcode_media']['video_view_count']

        item = {
                    "Link Video": link,
                    "Số View": view,
        }

        insta_list.append(item)
        time.sleep(.5)
    except:
        print(link + " Cannot get insta info!")
    
df = pd.DataFrame(insta_list)
df = insta_data.merge(df, on="Link Video", how="left")
df.to_pickle("insta.pkl")

In [249]:
insta_datetime = datetime.datetime.now()
print(insta_datetime)

2021-08-18 15:54:23.931543


# Facebook

In [251]:
fb_data = data[data['Link Video'].str.startswith('https://www.facebook.com/')]
fb_data = fb_data[fb_data['Link Video'].str.contains("/videos/")]

In [253]:
driver = webdriver.Chrome(chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.facebook.com/")

<ipython-input-253-fc674300ede4>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [254]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

fb_list = []

for link in fb_data["Link Video"].values:
 
    driver.get(link)
    
    try:

        view_count = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('//span[@class= "_26fq"]').text)

        if(view_count.find("triệu") == -1): # normal K count
            view = float(view_count[:-10].replace(",","."))
        else:
            view = int(view_count[:2]) * 1000 # million count, take 1 or 2 first digit times 10000000

        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        fb_list.append(item)
    
    except:
        print(link)
        
df = pd.DataFrame(fb_list)
df = fb_data.merge(df, on="Link Video", how="left")
df.to_pickle("fb.pkl")

fb_datetime = datetime.datetime.now()
print(fb_datetime)

2021-08-18 15:58:29.227112


In [255]:
tiktok = pd.read_pickle("tiktok.pkl")
insta = pd.read_pickle("insta.pkl")
fb = pd.read_pickle("fb.pkl")

In [256]:
tiktok_insta = pd.concat([tiktok, insta])
tiktok_insta["Số View"] = tiktok_insta["Số View"] / 1000
tiktok_insta_fb = pd.concat([tiktok_insta, fb])

In [ ]:
tiktok_insta_fb = tiktok_insta_fb[["index", "Số View"]]
export_df = data_from_excel.merge(tiktok_insta_fb, on="index", how="left")

In [267]:
time_df = pd.DataFrame({"Ngày giờ" :["Tiktok : " + str(Tiktok_time), "Insta : " + str(insta_datetime), "Fb : " + str(fb_datetime)]})
export_df = pd.concat([export_df, time_df], axis=1)
export_df.rename(columns={"Số View": "Số view (đơn vị 1000)"}, inplace=True)
push_df = export_df[["STT", "Mã INF", "Kênh hợp tác", "Link Video", "Người phụ trách", "Số view (đơn vị 1000)", "Ngày giờ"]]

In [268]:
final_push = push_df[~push_df.duplicated()]

In [28]:
# Export file to excel -> push it to gsheet
# export_df.to_excel("export_data.xlsx", index=False, sheet_name="View")

# Gsheet Export

In [275]:
import gspread
from gspread_dataframe import set_with_dataframe

# ACCES GOOGLE SHEET
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('Python') #->gg sheet name 

# APPEND DATA TO SHEET
your_dataframe = push_df
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET